<H2>課題5: 住宅価格の予測 (Advanced Version)</H2>
<p>課題2ではKaggleにある住宅価格予測を行いました。その際に利用したデータは、モデリングの基礎を学びやすくするために、変数の抜粋やデータの調整を施しました</p>
<p>では、調整されていない「実際のデータ」を使って解析するには、どうしたらよいでしょうか？Kaggle MasterのSerigne氏が、その方法をKaggleのサイト内で公開しています。<br>
https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard
</p>
<p>本課題では、上記のページに書かれている内容の中から重要な部分をピックアップし、<i>dataset/kaggle_housing_price.csv</i> のデータを使って作業に取り組んでもらいます。課題を通じて「実際のデータを使って解析する際にどのような作業を行うか」について学び、今までの知識をブラッシュアップしていきましょう。CSVの各列がどういう情報かについては、以下のページの Data fields を参照してください。<br>
https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data
</p>

<H2>1. 必要なモジュールの読み込み</H2>

In [1]:
import numpy as np
import pandas as pd 
import scipy as sp
from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import statsmodels.formula.api as smf
import statsmodels.api as sm

%matplotlib inline

<H2>2. データの読み込み </H2>

In [2]:
# データを読み込んで変数 data に格納


In [3]:
# データの最初の5行だけ表示


<H2>3. データ概要の確認</H2>
<p>そもそもt検定などを実施する場合、データに正規性と等分散性があるのか否かが重要です。等分散性がないと結果を正しく解析することができません（書籍では、等分散性を考慮しなくてもt検定が行える Welchの検定について触れています）。</p></p>
<p>正規性が確認できない場合は、何らかの **データ変換** を行い、正規性が確保できる状態にします（ただし、あらゆるデータ変換の方法を試すのはpハッキングになりかねませんので、気をつけましょう）。</p>

In [4]:
# Q-Qプロットを表示して正規性を確認


Q-Qプロットでデータの点が線上にきれいに乗っていれば、正規性は確認できます。もし、データの点が線から少し離れてしまっている（正規性が明確には確認できない）場合は、データ変換を試みます。今回は **対数変換** を行いましょう。

対数変換には、NumPyが持っている log1p を使います。NumPyの公式サイトに詳細が記載されています。<br>
https://docs.scipy.org/doc/numpy/reference/generated/numpy.log1p.html

また、対数変換を行う意味については、次のページなどで調べてみてください。<br>
https://atarimae.biz/archives/13161

In [5]:
#　NumPy の log1p を用いて全てのデータの対数変換を実施　


In [6]:
#　対数変換をした後のデータで再度Q-Qプロットを作成し、表示


対数変換しなかった場合よりも、データの点が線上にきれいに乗っているかと思います。これで正規性が確認できました。

<H2>4. 欠損値の確認と補充</H2>
<p>データセットが欠損値（空白）を含む場合、そのまま回帰分析等を行うと、エラーが発生したり、精度が悪くなったりします。</p>
<p>欠損値があまり多くない場合にはただ欠損値を含むデータを削除すればよいですが、あまりにも量が多い場合には削除してしまうとデータの多くを失ってしまうことになります。そのような場合には、**当たり障りのない値で埋める** という手法が取られます。</p>
<p><i>dataset/kaggle_housing_price.csv</i>では、欠損したデータには 'NA' という2文字が入っています。Pandasはデフォルトで 「'NA' とだけ入っているデータは欠損値である」というルールでCSVのデータを読み込んでくれますので、あとは私たちの手で欠損値（NA） をどう扱うか決めて、欠損したままにせず何らかのデータで補充する処理をしなくてはなりません。</p>
<p>なお、以下の作業では、書籍で紹介されていないPandasの関数を使います。以下の公式ドキュメントなどで調べながら取り組んでください。<br>
https://pandas.pydata.org/pandas-docs/version/0.23/api.html#dataframe
</p>

In [7]:
# 　以下の流れで、欠損値の数を調査して、表示してみましょう。
# 1. dataの各列に含まれる空白の数を調べ、割合を計算して、変数 all_data_na に格納する


# 2. all_data_na からPandasのDataFrameを作成し、変数 missing_data に格納する（割合の情報が入った列名は 'Missing_Ratio'にすること）


# 3. 空白が1つ以上ある列の情報のみを抽出して、表示


<p>次に、欠損値の補充を行います。本来なら全ての列について欠損値を埋めるべきですが、以下では2つの列についてのみ対応します。</p>
<H3>4-1. PoolQCの場合 </H3>
<p>プールが無い家の PoolQC（プールの品質） の列に NA が入っています。ですので「無いことを意味しているが欠損値として扱われないデータ」、たとえば "None" という文字列、もしくは整数の 0 で置き換えましょう。</p>

In [8]:
# PoolQCのNAを文字列 "None" で置換する


In [9]:
# PoolQCのNAが"None"で置換されたことを確認する


<H3>4-2. LotFrontageの場合</H3>
<p>LotFrontageは「間口の広さ（家屋の正面の幅）」です。LotFrontageが欠損している場合、どのような値を埋めれば良いでしょうか。一例としては「近隣の家々、つまり『近くの通りの名前（Neighborhood）』が同じ家々のLotFrontageの値とほぼ同じになる」という考え方です。そこで今回は、補充する値として「Neighborhoodが同じ家々のLotFrontafeの中央値（median）」を採用します。</p>

In [10]:
# LotFrontageのNAを、近隣の家（Neighborhood）単位でLotFrontageのmedianを取って、その値で埋める


In [11]:
# LotFrontageのNAが近隣の家々のLotFrontageのmedianで置換されたことを確認


<H2>5. 変数の選択</H2>
<p>テキストではあまり触れられていませんでしたが、相関関係のある変数を複数、モデルに含めてしまうと、回帰係数の分散が大きくなりモデルが不安定になります。この問題を **多重共線性** と言います。多重共線性を回避するためには、あらかじめ変数同士の相関について確認し、強い相関関係にあるものは取り除く必要があります。ここではその方法について確認しましょう。</p>
<p>その手法が、Kaggle内の以下のページにまとめられています。こちらの内容を参考にしながら、下記の作業に取り組んでください。<br>
https://www.kaggle.com/willkoehrsen/introduction-to-feature-selection
</p>

In [12]:
threshold = 0.8    # 相関係数が threshold に設定した数値以上だった場合、その変数を取り除く

In [13]:
# data から相関行列（Correlation Matrix）を作成して表示する。その際、絶対値を適用して負の値をプラスに変換すること


In [14]:
# 対角線上は絶対に 1 になること、および、対角線より下の部分は、上の部分と情報が重複していることから
# 相関行列を上三角行列に変換 


In [15]:
# 上三角行列から SalePrice以外の列について threshold 以上の値が1つ以上ある列を抽出。その「削除すべき変数」の名前および総数を表示する
# （→これを除去すべき変数と判断します）


In [16]:
# data から削除する変数を削除して data_dropped に格納し、data_dropped の内容を表示する


<H2>6. ダミー変数化</H2>
<p>カテゴリ変数は **ダミー変数化** する必要があります。ダミー変数の説明は、書籍では第5章の第2部（分散分析） 2-14で書かれていますので、そちらをご確認ください。</p>
<p>ひとつ例を出して説明します。たとえば、「Weather」という列があり、そこに入っているデータが、</p>

- 'Sunny'（晴れ）
- 'Cloudy'（くもり）
- 'Rainy'（雨）

<p>以上の3種類の文字列のみで構成されているとします。このとき「Weather」を、以下のような内容の3つの列に分割します。</p>

- Weather_Sunny：元の Weather のデータが 'Sunny' なら整数値の 1 が、それ以外なら整数値の 0 が入る
- Weather_Cloudy：元の Weather のデータが 'Cloudy' なら整数値の 1 が、それ以外なら整数値の 0 が入る
- Weather_Rainy：元の Weather のデータが 'Rainy' なら整数値の 1 が、それ以外なら整数値の 0 が入る

<p>このような作業がダミー変数化です。</p>

In [17]:
# data_dropped に含まれる全てのカテゴリ変数をダミー変数化して data_dummied に格納する。ダミー変数化には Pandas の関数を実行すること。
# （ひとつずつダミー変数化しても良いですが、ここでは一気に全てダミー変数化する命令を使います）


In [18]:
# ダミー変数化されたことを確認


<p>ここまで、読み込んだデータに対して「データの対数変換」「欠損値の補充」「多重共線性の回避」「ダミー変数化」の作業を行いました。このような作業を通したデータを分類や予測にかけると、精度の良い結果が得られます。</p>
<p>本課題としての必須項目は、ここまでです。お疲れ様でした！</p>
<p>もし余裕がある場合は、以下の（任意）の内容について取り組みましょう。</p>

<H2>7. （任意）住宅価格の予測の実行</H2>
<p>せっかくなので、得られたデータを予測にかけてみましょう。</p>

<h3>7-1. さらなる欠損値の補充</h3>
<p>上記では、欠損値の補充を2種類（PoolQC と LotFrontage）しか行いませんでした。まずは、変数 data に対してさらなる欠損値の補充を行いましょう。</p>

In [19]:
# https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard
# 上記ページの内容を参考に、data に入っているデータのうち、
# 既に補充している PoolQC と LotFrontage 以外の列について、補充すべき欠損値を補充する


<h3>7-2. 変数の選択とダミー変数化を再度実施する</h3>

In [20]:
threshold = 0.8    # 相関係数が threshold に設定した数値以上だった場合、その変数を取り除く

In [21]:
# data から相関行列（Correlation Matrix）を作成する。その際、絶対値を適用して負の値をプラスに変換すること


In [22]:
# 相関行列を上三角行列に変換 


In [23]:
# 「削除すべき変数」の名前および総数を表示する


In [24]:
# data から削除する変数を削除して data_dropped に格納し、data_dropped の内容を表示する


In [25]:
# data_dropped に含まれる全てのカテゴリ変数をダミー変数化して data_dummied に格納し、表示


<H3>7-3. データを訓練データとテストデータに分割</H3>

In [26]:
from sklearn.model_selection import train_test_split    # 必要なモジュールの読み込み

In [27]:
# 説明変数 X を用意する。ただし id列, SalePrice列は除去する


In [28]:
# 応答変数 y は SalePrice 列だけのデータで用意する


In [29]:
# X、y を訓練データ (X_train, y_train) とテストデータ (X_test, y_test) に分ける


<H3>7-4. 重回帰分析</H3>

In [30]:
from sklearn import linear_model    # 必要なモジュールの読み込み

In [31]:
# Ridge回帰モデルを作成して予測を実行する
# 1. alphaを50個つくる

# 2. RidgeCVを使って最適なalphaでモデルを構築する


# 3. 訓練データを指定してモデルを推定する


In [32]:
# 訓練データの的中率を表示


In [33]:
# テストデータの的中率を表示


データの件数を増やすなど工夫の余地はありますが、まずまずの的中率を得られたのではないでしょうか。